author: leezeeyee   
date: 2021/4/13  
link: [github](https://github.com/easilylazy/pattern-recognition) 

In [2]:
import pandas as pd


In [48]:
import random
import numpy as np

## Lecture 9编程作业
给定IRIS数据集做如下题目实验，该数据集有三类目标，每个类别有50个样本，每个样本有四维特征。实验时每个类别随机选30个样本进行训练，另外20个样本用于测试。  
1. 以感知器算法为基础分类算法，编写一个OVO多类分类器算法，对上述数据集进行实验，分析结果。    
2. 编写Softmax算法实现多类别分类，对上述数据集进行实验，分析结果。


In [6]:
irisData=pd.read_csv('iris.csv')

In [7]:
irisData.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [12]:
irisData['Species'][60]

'versicolor'

## types

In [13]:
setosas=irisData[irisData['Species']=='setosa']
versicolors=irisData[irisData['Species']=='versicolor']
virginicas=irisData[irisData['Species']=='virginica']

In [14]:
setosas.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [15]:
versicolors.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
50,51,7.0,3.2,4.7,1.4,versicolor
51,52,6.4,3.2,4.5,1.5,versicolor
52,53,6.9,3.1,4.9,1.5,versicolor
53,54,5.5,2.3,4.0,1.3,versicolor
54,55,6.5,2.8,4.6,1.5,versicolor


In [16]:
virginicas.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
100,101,6.3,3.3,6.0,2.5,virginica
101,102,5.8,2.7,5.1,1.9,virginica
102,103,7.1,3.0,5.9,2.1,virginica
103,104,6.3,2.9,5.6,1.8,virginica
104,105,6.5,3.0,5.8,2.2,virginica


## sample for train

In [19]:

def some(x, n):
    return x[random.sample(x.index, n)]

In [28]:
virginicas.index

Int64Index([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
            113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
            126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
            139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
           dtype='int64')

In [31]:
trainSetosas=setosas.sample(frac=0.6)
trainVersi=versicolors.sample(frac=0.6)
trainVirgin=virginicas.sample(frac=0.6)

In [58]:
trainData=[]
for train in [trainSetosas,trainVersi,trainVirgin]:
    values=train[['Sepal.Length',	'Sepal.Width',	'Petal.Length',	'Petal.Width']]
    trainData.append(np.asarray(values.iloc[:,:]))

## Train

In [61]:
def addSingleBias(X):
    N=X.shape[0]
    X=np.reshape(X,[N,1])
    bias_vector=np.ones([1,1])
    return np.row_stack((bias_vector,X))
def addBias(X):
    '''
    为维数为N,d的向量X添加偏置向量
    Args:
        X(numpy.ndarray):shape为[N,d]
    Returns:
        numpy.ndarray:shape为[N,d+1]，且第一列列向量全为1
    '''
    
    try:
        N=X.shape[0]
        d=X.shape[1]
        if d == 1:
            return addSingleBias(X)
        bias_vector=np.ones([N,1])
        return np.column_stack((bias_vector,X))
        
    except:
        return addSingleBias(X)

In [59]:
trainData=np.asarray(trainData)

In [62]:
trainData[0][:5]

array([[5.4, 3.9, 1.3, 0.4],
       [5.7, 3.8, 1.7, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2]])

## shuffle data

In [98]:
length=60
numArray=np.arange(length)
np.random.shuffle(numArray)

In [108]:
X1=addBias(trainData[0])
X2=addBias(trainData[1])
TrainX=np.row_stack([X1,X2])
TrainY=np.zeros(len(TrainX))
train=np.r_[X1,X2]
TrainY[:len(X1)]=1
TrainY[len(X1):]=-1
print(TrainX[25:35])
print(TrainY[25:35])
TrainX_sh=TrainX[numArray]
TrainY_sh=TrainY[numArray]
if np.not_equal(TrainX[numArray[0]],TrainX_sh[0]).any():
    print('error!')

[[1.  5.  3.3 1.4 0.2]
 [1.  5.2 3.5 1.5 0.2]
 [1.  5.1 3.7 1.5 0.4]
 [1.  4.4 3.  1.3 0.2]
 [1.  5.1 3.5 1.4 0.2]
 [1.  6.  2.7 5.1 1.6]
 [1.  5.6 3.  4.5 1.5]
 [1.  6.  2.2 4.  1. ]
 [1.  5.7 2.8 4.1 1.3]
 [1.  6.3 2.3 4.4 1.3]]
[ 1.  1.  1.  1.  1. -1. -1. -1. -1. -1.]


In [87]:
W=np.zeros([5,1])

In [92]:
foo=np.zeros([5,3])
bar=np.zeros([5,3])
np.row_stack([foo,bar])

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [88]:

for i in range(len(X1)):
    # print(X1[i])
    X=X1[i].reshape([5,1])
    # print(X)
    y=1
    if np.dot(W.transpose(),X)<=0:
        W_temp=W+y*X
        # print(W_temp)
        res=np.dot(X1,W)
        err1=len(res[res<=0])
        res=np.dot(X1,W_temp)
        err2=len(res[res<=0])
        print(err1)
        # print(err2)
        if err1>err2:
            W=W_temp
    
    res=np.dot(X1,W)
    err1=len(res[res<=0])
    print(err1)
    

30
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [51]:
foo=np.asarray(values.iloc[:,:])

In [52]:
foo

array([[5.4, 3.9, 1.3, 0.4],
       [5.7, 3.8, 1.7, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [4.8, 3.4, 1.9, 0.2],
       [4.9, 3.1, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [5.8, 4. , 1.2, 0.2],
       [4.6, 3.6, 1. , 0.2],
       [5.4, 3.7, 1.5, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [4.9, 3. , 1.4, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [4.6, 3.2, 1.4, 0.2],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.3, 0.3],
       [4.9, 3.6, 1.4, 0.1],
       [5.3, 3.7, 1.5, 0.2],
       [5. , 3.3, 1.4, 0.2],
       [5.2, 3.5, 1.5, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.4, 3. , 1.3, 0.2],
       [5.1, 3.5, 1.4, 0.2]])